##**DISPUTE HAPPENED IN INDIA**

## 1. Downloading packages and importing dataset


In [ ]:
!pip install pmdarima
!pip install tensorflow scikit-learn numpy pandas matplotlib
!pip install xgboost
!pip install lightgbm
!pip install prophet
!pip install folium
!pip install lifelines
!pip install pulp
!apt-get install -y glpk
!apt-get install -y coinor-cbc
!pip install gurobipy
!pip install pyomo
!pip install pgmpy
!pip install stable-baselines3 gym numpy
!pip install 'shimmy>=2.0'
!pip install stable-baselines3[extra] gymnasium pygame
!pip install dowhy
!pip install torch torchvision torchaudio pandas scikit-learn
!pip install tensorflow
!pip install geopandas
!pip install branca
!pip install sweetviz
!pip install pandas-profiling
!pip install -U pandas_profiling
!pip install ydata_profiling

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()
%matplotlib inline

LOADING DATASET

In [ ]:
conflict = pd.read_csv('https://raw.githubusercontent.com/bhadri-Raj-T/eda_project/refs/heads/main/dispute_dataset.csv')

#1. Data Pre-processing(Data Cleaning)

Making a copy

In [ ]:
conflict_copy=conflict.copy()

Checking missing data

In [ ]:
miss=conflict_copy.isnull().sum()
miss1 = (conflict_copy.isnull().sum()/len(conflict))* 100
m = pd.concat([miss,miss1],axis=1,keys=['Total','Missing%'])
m

- From the above output we can see that __director__ , __cast__ ,**country** columns contains __maximum null values__. We will see how to deal with them.

- So, We Delete director and cast columns because they are not going to use those features right now.

Visualizing missing data

In [ ]:
sns.heatmap(conflict_copy.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Data Heatmap")
plt.show()

Handling missing data

In [ ]:
conflict_copy = conflict_copy.dropna(axis=1, thresh=0.7 * len(conflict_copy))
conflict_copy = conflict_copy.dropna(axis=0, subset=['conflict_name', 'type_of_violence'])

In [ ]:
conflict_copy['deaths_a'].fillna(conflict_copy['deaths_a'].median(), inplace=True)
conflict_copy['deaths_b'].fillna(0, inplace=True)
conflict_copy['source_original'].fillna('Unknown', inplace=True)
conflict_copy['where_description'].fillna('No Description', inplace=True)
conflict_copy['adm_1'].fillna('Unknown', inplace=True)
conflict_copy['adm_2'].fillna('Unknown', inplace=True)

In [ ]:
conflict_copy.isnull().sum()

Converting data types

In [ ]:
conflict_copy['id'] = conflict_copy['id'].astype(int)
conflict_copy['conflict_dset_id'] = conflict_copy['conflict_dset_id'].astype(int)
conflict_copy['conflict_new_id'] = conflict_copy['conflict_new_id'].astype(int)
conflict_copy['dyad_dset_id'] = conflict_copy['dyad_dset_id'].astype(int)
conflict_copy['dyad_new_id'] = conflict_copy['dyad_new_id'].astype(int)
conflict_copy['side_a_dset_id'] = conflict_copy['side_a_dset_id'].astype(int)
conflict_copy['side_a_new_id'] = conflict_copy['side_a_new_id'].astype(int)
conflict_copy['side_b_dset_id'] = conflict_copy['side_b_dset_id'].astype(int)
conflict_copy['side_b_new_id'] = conflict_copy['side_b_new_id'].astype(int)
conflict_copy['priogrid_gid'] = conflict_copy['priogrid_gid'].astype(int)
conflict_copy['country_id'] = conflict_copy['country_id'].astype(int)

In [ ]:
conflict_copy['type_of_violence'] = conflict_copy['type_of_violence'].astype('category')
conflict_copy['event_clarity'] = conflict_copy['event_clarity'].astype('category')
conflict_copy['date_prec'] = conflict_copy['date_prec'].astype('category')

In [ ]:
conflict_copy['date_start'] = pd.to_datetime(conflict_copy['date_start'], errors='coerce')
conflict_copy['date_end'] = pd.to_datetime(conflict_copy['date_end'], errors='coerce')

In [ ]:
count_columns = ['number_of_sources', 'deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best', 'high', 'low']
conflict_copy[count_columns] = conflict_copy[count_columns].astype(int)

In [ ]:
categorical_cols = ['code_status', 'conflict_name', 'dyad_name', 'side_a', 'side_b', 'country', 'iso3', 'region']
conflict_copy[categorical_cols] = conflict_copy[categorical_cols].astype('category')

In [ ]:
conflict_copy['active_year'] = pd.to_numeric(conflict_copy['year'], errors='coerce')

In [ ]:
conflict_copy['latitude'] = pd.to_numeric(conflict_copy['latitude'], errors='coerce')
conflict_copy['longitude'] = pd.to_numeric(conflict_copy['longitude'], errors='coerce')

Log Transformation of Death Counts

In [ ]:
# Apply log transformation (add 1 to avoid log(0))
conflict_copy[['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown']] = \
    np.log1p(conflict_copy[['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown']])

Cleaning and Filtering Conflict Data

In [ ]:
# 1️⃣ Drop the first row (it contains headers, not real data)
df=conflict.copy()
df_clean = df.iloc[1:].copy()

# 2️⃣ Convert data types
df_clean["year"] = pd.to_numeric(df_clean["year"], errors='coerce')
df_clean["latitude"] = pd.to_numeric(df_clean["latitude"], errors='coerce')
df_clean["longitude"] = pd.to_numeric(df_clean["longitude"], errors='coerce')
df_clean["best"] = pd.to_numeric(df_clean["best"], errors='coerce')
df_clean["date_start"] = pd.to_datetime(df_clean["date_start"], errors='coerce')
df_clean["date_end"] = pd.to_datetime(df_clean["date_end"], errors='coerce')

# 3️⃣ Filter for only India
df_clean = df_clean[df_clean["country"] == "India"].copy()

# 4️⃣ Drop columns with more than 50% missing values
missing_threshold = 0.5
cols_to_drop = df_clean.columns[df_clean.isnull().mean() > missing_threshold]
df_clean.drop(columns=cols_to_drop, inplace=True)

# 5️⃣ Drop rows with missing essential fields
critical_cols = ["latitude", "longitude", "year", "best", "type_of_violence", "date_start", "date_end"]
df_clean.dropna(subset=critical_cols, inplace=True)

# 6️⃣ Fill non-critical missing fields with placeholders
df_clean.fillna({
    "adm_1": "Unknown",
    "adm_2": "Unknown",
    "where_description": "Unknown",
    "side_a": "Unknown",
    "side_b": "Unknown"
}, inplace=True)

# ✅ Summary after cleaning
print("\nCleaned Data Shape:", df_clean.shape)

In [ ]:
df1=conflict_copy.copy()

In [ ]:
df1.rename(columns={'best': 'best_est'}, inplace=True)
df1['date_start'] = pd.to_datetime(df1['date_start'], errors='coerce')
df1['date_end'] = pd.to_datetime(df1['date_end'], errors='coerce')
df1['year'] = df1['year'].astype('str')
df1['year'] = df1['year'].astype('int')
# Convert important columns to numeric
numeric_cols = [
    'best_est', 'deaths_a', 'deaths_b',
    'deaths_civilians', 'deaths_unknown',
    'latitude', 'longitude'
]

# Convert each column to float
for col in numeric_cols:
    df1[col] = pd.to_numeric(df1[col], errors='coerce')
df1['event_duration'] = (df1['date_end'] - df1['date_start']).dt.days
df1['location_precision'] = df1['where_prec'].map({
    1: 'Exact', 2: 'Near', 3: 'ADM2', 4: 'ADM1',
    5: 'BroadArea', 6: 'CountryOnly', 7: 'Unknown'
})
# Drop rows with missing target or year
df1.dropna(subset=['best_est', 'year'], inplace=True)

# Optional: fill or drop missing locations
df1['adm_1'].fillna("Unknown", inplace=True)
df1['adm_2'].fillna("Unknown", inplace=True)
df1['where_description'].fillna("Unknown", inplace=True)

# Drop duplicates
df1.drop_duplicates(inplace=True)
from scipy.stats import zscore

# Compute z-scores of the target variable
df1['zscore_best_est'] = zscore(df1['best_est'])

# Remove extreme outliers (z > 3 or z < -3)
df1 = df1[(df1['zscore_best_est'] < 3) & (df1['zscore_best_est'] > -3)]

# Drop the zscore column
df1.drop(columns='zscore_best_est', inplace=True)
df1.drop(columns=['geom_wkt', 'relid', 'code_status'], inplace=True)

#2. Data Profiling (Descriptive Analysis)

Visualize missing values

In [ ]:
# Visualize missing values
sns.heatmap(conflict_copy.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Data Heatmap")
plt.show()

Displaying the First 5 Rows of the DataFrame

In [ ]:
conflict_copy.head()

Displaying the Last 5 Rows of the DataFrame

In [ ]:
conflict_copy.tail()

Checking the shape of the dataset

In [ ]:
conflict_copy.shape

 Checking the Total Number of Elements in the Dataset

In [ ]:
conflict_copy.size

Listing All Column Names in the Dataset


In [ ]:
conflict_copy.columns

Checking Data Types of Each Column in the Dataset

In [ ]:
conflict_copy.dtypes

Displaying Dataset Information Including Non-Null Counts and Data Types

In [ ]:
conflict_copy.info()

Statistics of the dataset

In [ ]:
conflict_copy.describe()

Generating Descriptive Statistics for All Columns in the Dataset

In [ ]:
conflict_copy.describe(include='all')

Checking the Number of Unique Values in Each Column

In [ ]:
conflict_copy.nunique()

Catagorizing the data

In [ ]:
categorical_cols = ["code_status","type_of_violence","active_year", "where_prec","conflict_name","dyad_name","side_a","side_b","source_article","source_original","where_coordinates","where_description","adm_1","adm_2","geom_wkt","country","iso3","region","event_clarity","date_prec",]  # Example categorical columns
numerical_cols = ["latitude", "longitude", "deaths_a", "deaths_b", "deaths_civilians", "deaths_unknown", "best", "high", "low","number_of_sources","priogrid_gid"]

**Univariate Analysis**

In [ ]:
# Distribution of 'type_of_violence'
sns.countplot(x='type_of_violence', data=conflict_copy)
plt.title("Distribution of Type of Violence")
plt.show()

Histogram for Deaths (Side A)

In [ ]:
# Histogram for 'deaths_a'
conflict_copy['deaths_a'].hist(bins=50, edgecolor='black')
plt.title("Distribution of Deaths (Side A)")
plt.xlabel("Deaths (Side A)")
plt.ylabel("Frequency")
plt.show()

**Visualizing Conflict Locations on a Map**

In [ ]:
import folium

# Create a map centered at the average location
m = folium.Map(location=[conflict_copy['latitude'].mean(), conflict_copy['longitude'].mean()], zoom_start=2)

# Add markers for each conflict location
for _, row in conflict_copy.iterrows():
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=3,
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(m)

m


**Checking for Anomalies and Outliers**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

numerical_cols = conflict_copy.select_dtypes(include=['float64', 'int64']).columns

for col in numerical_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=conflict_copy[col])
    plt.title(f"Boxplot for {col}")
    plt.show()

**Relationship Between Variables**

**Scatter Plots**

In [ ]:
# Scatter plot for numerical relationships
plt.figure(figsize=(8, 6))
sns.scatterplot(x='deaths_a', y='deaths_civilians', data=conflict_copy, hue='type_of_violence')
plt.title("Deaths (Side A) vs. Civilian Deaths")
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(x='latitude', y='longitude', data=conflict_copy, hue='region')
plt.title("Geographical Spread of Conflicts by Region")
plt.show()


Pairplot for Numerical Columns

In [ ]:
sns.pairplot(conflict_copy, vars=['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown'], hue='type_of_violence')
plt.show()
sns.pairplot(conflict_copy, vars=numerical_cols, hue='type_of_violence')
plt.show()


**Time-Series Analysis**

**Conflict Trends Over the Years**

In [ ]:
conflicts_by_year = conflict.groupby('year').size()

plt.figure(figsize=(10, 5))
conflicts_by_year.plot(kind='line', title='Number of Conflicts Over the Years', marker='o')
plt.xlabel('Year')
plt.ylabel('Number of Conflicts')
plt.grid()
plt.show()


**Casualties Over Time**

In [ ]:
# Aggregate casualties by year
casualties_by_year = conflict.groupby('year')[['deaths_a', 'deaths_b', 'deaths_civilians']].sum()

# Plot casualties over time
casualties_by_year.plot(kind='line', figsize=(12, 6), title='Casualties Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Casualties')
plt.legend(['Deaths Side A', 'Deaths Side B', 'Civilian Deaths'])
plt.grid()
plt.show()


skewness and kurtosis

In [ ]:
conflict_copy[numerical_cols].skew()

In [ ]:
conflict_copy[numerical_cols].kurtosis()

Aggregate yearly statistics

In [ ]:
violence_type_map = {1: "State-based", 2: "Non-state", 3: "One-sided"}
df_clean["violence_type"] = df_clean["type_of_violence"].map(violence_type_map)

yearly_summary = df_clean.groupby(["year", "violence_type"]).agg(
    total_events=("id", "count"),
    total_fatalities=("best", "sum")
).reset_index()

Total Conflict Events per Year in India by Conflict Type

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=yearly_summary, x="year", y="total_events", hue="violence_type", marker="o")
plt.title("Total Conflict Events per Year in India by Conflict Type")
plt.xlabel("Year")
plt.ylabel("Number of Events")
plt.grid(True)
plt.tight_layout()
plt.show()

Total Fatalities Per Year

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=yearly_summary, x="year", y="total_fatalities", hue="violence_type", marker="o")
plt.title("Total Fatalities per Year in India by Conflict Type")
plt.xlabel("Year")
plt.ylabel("Number of Fatalities (Best Estimate)")
plt.grid(True)
plt.tight_layout()
plt.show()

Top 10 conflict-heavy states

In [ ]:
state_summary = df_clean.groupby("adm_1").agg(
    total_events=("id", "count"),
    total_fatalities=("best", "sum")
).sort_values(by="total_fatalities", ascending=False).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(data=state_summary.head(10), x="total_fatalities", y="adm_1", palette="Reds_r")
plt.title("Top 10 Conflict-Hit States by Fatalities")
plt.xlabel("Total Fatalities")
plt.ylabel("State (adm_1)")
plt.tight_layout()
plt.show()


Visual representation of conflicts on map\
Try zooming and check the map!

In [ ]:
import folium
from folium.plugins import MarkerCluster

# 📍 Create a folium map centered on India
india_map = folium.Map(location=[22.5937, 78.9629], zoom_start=5, tiles='CartoDB positron')

# 📍 Cluster markers for conflicts
marker_cluster = MarkerCluster().add_to(india_map)

# Add markers
for idx, row in df_clean.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=3,
        color="red",
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(f"{row['conflict_name']}<br>Fatalities: {row['best']}", max_width=200)
    ).add_to(marker_cluster)

# Display the map
india_map


MAP heatmap

In [ ]:
import folium
from folium.plugins import HeatMap
import branca.colormap as cm

# 🌍 Create base map
heatmap_map = folium.Map(location=[22.9734, 78.6569], zoom_start=5, tiles="CartoDB positron")

# 🧪 Prepare heatmap data
heat_data = [[row["latitude"], row["longitude"], row["best"]] for index, row in df_clean.iterrows() if not np.isnan(row["best"])]

# 🔥 Add heatmap
HeatMap(heat_data, radius=10, blur=15, max_zoom=10).add_to(heatmap_map)

# 🎨 Add custom color legend
colormap = cm.LinearColormap(
    colors=["green", "yellow", "orange", "red"],
    vmin=df_clean["best"].min(),
    vmax=df_clean["best"].max(),
    caption="Conflict Fatalities (Best Estimate)"
)
colormap.add_to(heatmap_map)

# Show the map
heatmap_map


Top Conflict States Colored by Fatality Range

In [ ]:
bins = [0, 10, 100, 500, 1000, df_clean["best"].max()]
labels = ["0-10", "11-100", "101-500", "501-1000", "1000+"]
df_clean["fatality_range"] = pd.cut(df_clean["best"], bins=bins, labels=labels)

# Barplot with color by range
plt.figure(figsize=(12, 6))
sns.countplot(data=df_clean, y="adm_1", hue="fatality_range", order=df_clean["adm_1"].value_counts().index[:10])
plt.title("Top Conflict States Colored by Fatality Range")
plt.xlabel("Number of Events")
plt.ylabel("State")
plt.legend(title="Fatality Range")
plt.tight_layout()
plt.show()


Top 10 most affected districts

In [ ]:
district_summary = df_clean.groupby("adm_2").agg(
    total_events=("id", "count"),
    total_fatalities=("best", "sum")
).sort_values(by="total_fatalities", ascending=False).reset_index()


plt.figure(figsize=(12, 6))
sns.barplot(data=district_summary.head(10), x="total_fatalities", y="adm_2", palette="Oranges_r")
plt.title("Top 10 Conflict-Hit Districts by Fatalities")
plt.xlabel("Total Fatalities")
plt.ylabel("District (adm_2)")
plt.tight_layout()
plt.show()


Plot Top 10 dyads by fatalities

In [ ]:
# 🎯 Top Actors (side A and B) by number of events
top_side_a = df_clean["side_a"].value_counts().head(10)
top_side_b = df_clean["side_b"].value_counts().head(10)

# 🔥 Fatalities by Dyad (interaction)
dyad_fatalities = df_clean.groupby("dyad_name").agg(
    total_events=("id", "count"),
    total_fatalities=("best", "sum")
).sort_values(by="total_fatalities", ascending=False).reset_index()


plt.figure(figsize=(12, 6))
sns.barplot(data=dyad_fatalities.head(10), x="total_fatalities", y="dyad_name", palette="Blues_r")
plt.title("Top 10 Actor Dyads by Conflict Fatalities")
plt.xlabel("Total Fatalities")
plt.ylabel("Actor Dyad")
plt.tight_layout()
plt.show()


Top side_a actors (e.g., Government of India, Army, etc.)

In [ ]:

side_a_stats = df_clean.groupby("side_a").agg(
    total_events=("id", "count"),
    total_fatalities=("best", "sum"),
    civilian_deaths=("deaths_civilians", "sum")
).sort_values(by="total_fatalities", ascending=False).reset_index()

# Top 10
plt.figure(figsize=(12, 6))
sns.barplot(data=side_a_stats.head(10), x="total_fatalities", y="side_a", palette="Reds_r")
plt.title("Top State Forces (Side A) by Total Fatalities")
plt.xlabel("Total Fatalities")
plt.ylabel("Side A (State Forces)")
plt.tight_layout()
plt.show()


Top side_b actors (e.g., Maoists, ULFA, LeT)

In [ ]:

side_b_stats = df_clean.groupby("side_b").agg(
    total_events=("id", "count"),
    total_fatalities=("best", "sum"),
    civilian_deaths=("deaths_civilians", "sum")
).sort_values(by="total_fatalities", ascending=False).reset_index()

# Top 10
plt.figure(figsize=(12, 6))
sns.barplot(data=side_b_stats.head(10), x="total_fatalities", y="side_b", palette="magma")
plt.title("Top Insurgent Groups (Side B) by Total Fatalities")
plt.xlabel("Total Fatalities")
plt.ylabel("Side B (Non-State Groups)")
plt.tight_layout()
plt.show()


Civilian harm by side_a

In [ ]:
side_a_civilians = side_a_stats.sort_values(by="civilian_deaths", ascending=False).head(20)
side_a_civilians[["side_a", "total_events", "civilian_deaths"]]

Civilian harm by side_b

In [ ]:
side_b_civilians = side_b_stats.sort_values(by="civilian_deaths", ascending=False).head(20)
side_b_civilians[["side_b", "total_events", "civilian_deaths"]]


#3. Diagnosis analysis


**Z-Score Analysis**

In [ ]:
from scipy.stats import zscore

z_scores = conflict_copy[numerical_cols].apply(zscore)

outliers = conflict_copy[(z_scores.abs() > 3).any(axis=1)]
print("\nPotential Outliers:")
outliers


**Correlation Analysis**

In [ ]:
# Increase figure size
plt.figure(figsize=(12, 8))  # Adjust width and height as needed

# Correlation heatmap
corr = conflict_copy[numerical_cols].corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm',linewidths=0.5)

plt.title("Correlation Heatmap", fontsize=14)
plt.show()


duplicates

In [ ]:
conflict_copy.duplicated().sum()

Find the frequency

In [ ]:
for i in categorical_cols:
  print(conflict_copy[i].value_counts())

Histograms for deaths

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 15))
sns.histplot(conflict_copy["deaths_a"], bins=5, kde=True, ax=axes[0, 0])
axes[0, 0].set_title("Distribution of Deaths on Side A")

sns.histplot(conflict_copy["deaths_b"], bins=5, kde=True, ax=axes[0, 1])
axes[0, 1].set_title("Distribution of Deaths on Side B")

sns.histplot(conflict_copy["best"], bins=5, kde=True, ax=axes[0, 2])
axes[0, 2].set_title("Distribution of Best Estimate of Deaths")

sns.boxplot(y=conflict_copy["latitude"], ax=axes[1, 0])
axes[1, 0].set_title("Boxplot of Latitude")

sns.boxplot(y=conflict_copy["longitude"], ax=axes[1, 1])
axes[1, 1].set_title("Boxplot of Longitude")

sns.boxplot(y=conflict_copy["best"], ax=axes[1, 2])
axes[1, 2].set_title("Boxplot of Best Estimate of Deaths")

MEANS

In [ ]:
# Mean fatalities by violence type
print(df_clean.groupby("type_of_violence")["best"].mean())
print()
# Mean fatalities by region
print(df_clean.groupby("region")["best"].mean())
print()
# Most lethal side_b (insurgent) groups
print(df_clean.groupby("side_b")["best"].sum().sort_values(ascending=False).head(10))
print()
# Most lethal side_a (insurgent) groups
print(df_clean.groupby("side_a")["best"].sum().sort_values(ascending=False).head(10))

Impact Assesment and Clustering

In [ ]:
import matplotlib.pyplot as plt

# Total deaths by category
death_totals = {
    "Civilians": df["deaths_civilians"].sum(),
    "Side A (Govt forces)": df["deaths_a"].sum(),
    "Side B (Insurgents)": df["deaths_b"].sum()
}

plt.figure(figsize=(8, 5))
plt.bar(death_totals.keys(), death_totals.values(), color=["skyblue", "orange", "tomato"])
plt.title("Total Deaths by Conflict Participant Type")
plt.ylabel("Number of Fatalities")
plt.grid(axis='y')
plt.tight_layout()
plt.show()


Year-wise Civilian Fatalities

In [ ]:
df=df1
df['year'] = df['date_start'].dt.year

civilian_impact = df.groupby("year")["deaths_civilians"].sum()

plt.figure(figsize=(10, 5))
plt.plot(civilian_impact.index, civilian_impact.values, marker='o', color='crimson')
plt.title("Year-wise Civilian Fatalities")
plt.xlabel("Year")
plt.ylabel("Number of Civilian Deaths")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df=df1
type_impact = df.groupby("type_of_violence")[["best_est", "deaths_civilians"]].mean().rename({
    "best": "Avg Total Fatalities",
    "deaths_civilians": "Avg Civilian Fatalities"
}, axis=1)

print(type_impact)



Civilian Fatalities Over Time by Type of Violence

In [ ]:
# Group and pivot the data
df['year'] = df['date_start'].dt.year

civilian_by_type = df.groupby(["year", "type_of_violence"])["deaths_civilians"].sum().unstack(fill_value=0)

# Optional: Rename types for clarity
violence_labels = {
    1: "State-based",
    2: "Non-state",
    3: "One-sided"
}
civilian_by_type.rename(columns=violence_labels, inplace=True)

# Plot (stacked area or line)
civilian_by_type.plot(kind='area', stacked=True, figsize=(12, 6), colormap='tab10')
plt.title("Civilian Fatalities Over Time by Type of Violence")
plt.xlabel("Year")
plt.ylabel("Civilian Deaths")
plt.grid(True)
plt.legend(title="Type of Violence")
plt.tight_layout()
plt.show()


Proportion of Side A Fatalities Caused by Side B Groups

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Group by side_b and sum side_a deaths
side_b_impact = df.groupby("side_b")["deaths_a"].sum().sort_values(ascending=False)

# Filter out 0-death entries
side_b_impact = side_b_impact[side_b_impact > 0]

# Take top N groups, rest as 'Others'
top_n = 6
top_groups = side_b_impact[:top_n]
other = side_b_impact[top_n:].sum()

# ✅ Combine using concat instead of append
side_b_final = pd.concat([top_groups, pd.Series({"Others": other})])

# Pie chart
plt.figure(figsize=(8, 8))
side_b_final.plot(kind='pie', autopct='%1.1f%%', startangle=140, colormap='tab10', textprops={'fontsize': 10})
plt.title("Proportion of Side A Fatalities Caused by Side B Groups")
plt.ylabel("")  # Hide y-axis label
plt.tight_layout()
plt.show()


Clustering


In [ ]:
from sklearn.preprocessing import StandardScaler

features = ["deaths_a", "deaths_b", "deaths_civilians"]
df_cluster = df[features].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster)

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, 10), wcss, marker='o')
plt.title("Elbow Method - Optimal Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("WCSS")
plt.grid(True)
plt.tight_layout()
plt.show()


 Performing K-Means Clustering to Identify Patterns in the Data


In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# See cluster breakdown
print(df['cluster'].value_counts())


Visualizing Clusters Using PCA (Principal Component Analysis)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce to 2 principal components for 2D visualization
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)

# Add to DataFrame for plotting
df['pca1'] = pca_result[:, 0]
df['pca2'] = pca_result[:, 1]

# Plot clusters
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['pca1'], df['pca2'], c=df['cluster'], cmap='tab10', alpha=0.6)
plt.title("Clustering of Conflict Events Based on Fatalities")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.legend(*scatter.legend_elements(), title="Cluster")
plt.grid(True)
plt.tight_layout()
plt.show()


Analyzing Cluster Profiles by Calculating the Mean of Features

In [ ]:
# Group by cluster and calculate mean of features
cluster_summary = df.groupby("cluster")[["deaths_a", "deaths_b", "deaths_civilians"]].mean()
print(" Cluster Profiles:")
print(cluster_summary)


A cluster with high deaths_b → intense insurgent targeting

High deaths_a → state retaliation/failure

High deaths_civilians → violent civilian-targeted events

Cluster 0:  Low-intensity conflicts(Small-scale, frequent events with minor casualties)

Cluster 1:  Government-targeted massacre

Cluster 2: Major armed conflicts(Extremely high insurgent fatalities, some state losses, no civilian impact — likely major military ops)


In [ ]:
conflict_type_map = {1: "State-based", 2: "Non-state", 3: "One-sided"}
df["violence_type_name"] = df["type_of_violence"].map(conflict_type_map)

# Count per cluster & violence type
ct_counts = df.groupby(["cluster", "violence_type_name"]).size().unstack(fill_value=0)
print(ct_counts)

# Plot
ct_counts.plot(kind='bar', stacked=True, figsize=(10, 5), colormap='Set3')
plt.title("Distribution of Conflict Types per Cluster")
plt.ylabel("Number of Events")
plt.xlabel("Cluster")
plt.legend(title="Type of Violence")
plt.tight_layout()
plt.show()


### Identifying Top States in Each Cluster Based on Conflict Distribution


In [ ]:
# Top states in each cluster
location_distribution = df.groupby(["cluster", "adm_1"]).size().unstack(fill_value=0)

# Show top 5 locations in each cluster
print("Top States in Each Cluster:")
for cluster_id in location_distribution.index:
    top_states = location_distribution.loc[cluster_id].sort_values(ascending=False).head(5)
    print(f"\nCluster {cluster_id}:")
    print(top_states)

### Analyzing Year-wise Distribution of Conflict Clusters


In [ ]:
import geopandas as gpd
# Assuming you have year-wise data tagged with cluster labels
df.groupby(['year', 'cluster']).size().unstack().plot()
plt.title("Year-wise Distribution of Conflict Clusters")
plt.ylabel("Number of Events")
plt.show()


Clustering using Hierarchical

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
import folium

df=conflict_copy
# Assuming your preprocessed features are in a matrix `X` (e.g., after scaling/encoding)
# Example: Select numerical features (adjust as needed)
num_features = ['deaths_a', 'deaths_b', 'deaths_civilians', 'best', 'latitude', 'longitude']
X = df[num_features]

# Standardize features (critical for hierarchical clustering)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform hierarchical clustering
Z = linkage(X_scaled, method='ward')  # 'ward' minimizes variance; alternatives: 'complete', 'average'

# Plot the dendrogram to decide number of clusters
plt.figure(figsize=(12, 6))
dendrogram(Z, truncate_mode='lastp', p=20, show_leaf_counts=True)
plt.title('Dendrogram (Hierarchical Clustering)')
plt.xlabel('Sample Index or (Cluster Size)')
plt.ylabel('Distance (Ward)')
plt.axhline(y=15, color='r', linestyle='--')  # Adjust line to find clusters (e.g., where vertical lines intersect)
plt.show()

# Choose a distance threshold or number of clusters
distance_threshold = 15  # Adjust based on dendrogram
n_clusters = 4  # Or use threshold

# Assign clusters to data points
clusters = fcluster(Z, t=distance_threshold, criterion='distance')  # or `t=n_clusters, criterion='maxclust'`
df['cluster'] = clusters

# Analyze cluster characteristics
cluster_summary = df.groupby('cluster')[num_features].mean()
cluster_summary

In [ ]:
# Define merge rules (example: group clusters with similar death profiles)
merge_rules = {
    # Low-intensity state conflicts (low deaths_a, near-zero deaths_b)
    1: 1, 2: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 17: 1, 18: 1,

    # High-intensity symmetric (balanced deaths_a and deaths_b)
    3: 2, 4: 2, 23: 2, 39: 2, 40: 2, 41: 2, 48: 2, 51: 2, 52: 2,

    # Civilian-targeting (high deaths_civilians)
    26: 3, 27: 3, 28: 3, 30: 3, 31: 3, 42: 3, 43: 3,

    # Extreme events (outliers with massive fatalities)
    24: 4, 25: 4, 36: 4, 37: 4, 50: 4,

    # Others (default to a new cluster)
    **{k: 5 for k in
    range(16, 52) if k not in [16, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]}
}

df['cluster'] = df['cluster'].replace(merge_rules)
cluster_names = {
    1: 'Low-Intensity State Conflicts',
    2: 'High-Intensity Symmetric Wars',
    3: 'Civilian-Targeting Atrocities',
    4: 'Mass-Fatality Extreme Events',
    5: 'Other Asymmetric Conflicts'
}
df['cluster_name'] = df['cluster'].map(cluster_names)
unmapped_clusters = set(df['cluster'].unique()) - set(merge_rules.keys())
print(f"Unmapped clusters: {unmapped_clusters}")
# Get all original cluster IDs (1-52)
all_clusters = set(range(1, 53))

# Assign unmapped clusters to "Other Asymmetric Conflicts" (cluster 5)
merge_rules_complete = {**merge_rules, **{k: 5 for k in all_clusters if k not in merge_rules}}
df['cluster'] = df['cluster'].replace(merge_rules_complete)
df['cluster_name'] = df['cluster'].map(cluster_names)

# Verify no NaN values remain
assert df['cluster_name'].isna().sum() == 0, "Still missing some cluster mappings!"

In [ ]:
cluster_summary = df.groupby('cluster_name')[num_features].mean()
cluster_summary

In [ ]:
# Color mapping for new clusters
color_map = {
    'Low-Intensity State Conflicts': 'green',
    'High-Intensity Symmetric Wars': 'blue',
    'Civilian-Targeting Atrocities': 'red',
    'Mass-Fatality Extreme Events': 'black',
    'Other Asymmetric Conflicts': 'orange'
}

# Create Folium map
map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=np.log(row['best'] + 1) * 2,  # Scale by deaths
        color=color_map[row['cluster_name']],
        fill=True,
        tooltip=f"{row['cluster_name']}: Best={row['best']:.0f}"
    ).add_to(map)
map.save('simplified_clusters.html')
map

In [ ]:
map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=5)

for _, row in df.iterrows():
    # Custom popup HTML with cluster details
    popup_html = f"""
    <b>Cluster {row['cluster']}</b><br>
    <b>Type:</b> {row['cluster_name']}<br>
    <b>Deaths (Best):</b> {row['best']:.0f}<br>
    <b>Side A Deaths:</b> {row['deaths_a']:.1f}<br>
    <b>Side B Deaths:</b> {row['deaths_b']:.1f}<br>
    <b>Civilians:</b> {row['deaths_civilians']:.1f}
    """

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=np.log(row['best'] + 1) * 2,
        color=color_map[row['cluster_name']],
        fill=True,
        popup=folium.Popup(popup_html, max_width=250),  # Popup shows on click
        tooltip=f"Cluster {row['cluster']}"  # Tooltip on hover
    ).add_to(map)

map.save("map_with_interactive_labels.html")
map

DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
df = conflict_copy  # Replace with your dataset

# Selecting relevant features for clustering
X = df[['latitude', 'longitude']]

# Standardize data for better clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.3, min_samples=5)  # Adjust eps and min_samples based on data density
df['cluster'] = dbscan.fit_predict(X_scaled)
plt.figure(figsize=(8, 5))
plt.scatter(df['latitude'], df['longitude'], c=df['cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('DBSCAN - High-Density Conflict Areas')
plt.show()

# Remove noise points (cluster = -1)
df_filtered = df[df['cluster'] != -1]

# Plot results after noise removal
plt.figure(figsize=(8, 5))
plt.scatter(df_filtered['latitude'], df_filtered['longitude'], c=df_filtered['cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('DBSCAN - High-Density Conflict Areas (Noise Removed)')
plt.show()

# Count clusters and removed noise points
num_clusters = len(set(df_filtered['cluster']))
num_noise_points = len(df) - len(df_filtered)

print(f"Number of Clusters: {num_clusters}")
print(f"Number of Noise Points Removed: {num_noise_points}")

Kaplan-Meier Estimator (for time-to-event analysis)


In [ ]:
from lifelines import KaplanMeierFitter

# Load your dataset (Ensure it has a 'duration' column in days/months/years)
df = conflict_copy  # Replace with actual dataset

# Calculate conflict duration (days)
df['duration'] = (df['date_end'] - df['date_start']).dt.days

# Censorship column: 1 if conflict ended, 0 if still ongoing
df['event_observed'] = df['date_end'].notna().astype(int)

# Kaplan-Meier model
kmf = KaplanMeierFitter()

# Fit the model
kmf.fit(df['duration'], event_observed=df['event_observed'])

# Plot the survival function
plt.figure(figsize=(8, 5))
kmf.plot()
plt.xlabel("Days since Conflict Start")
plt.ylabel("Survival Probability")
plt.title("Kaplan-Meier Survival Curve for Conflicts")
plt.show()

Cox Proportional Hazards Model (for multivariate analysis)

In [ ]:
from lifelines import CoxPHFitter

# Load dataset (Make sure it has 'start_date', 'end_date', and relevant features)
df = conflict_copy

# Select features for multivariate analysis
features = ['type_of_violence', 'deaths_a', 'deaths_b', 'event_clarity', 'number_of_sources']

# Drop rows with missing values
df = df.dropna(subset=['duration', 'event_observed'] + features)

# Prepare dataset for Cox model
cox_data = df[['duration', 'event_observed'] + features]

# Fit Cox Proportional Hazards model
cph = CoxPHFitter()
cph.fit(cox_data, duration_col='duration', event_col='event_observed')

# Summary of the model
cph.print_summary()

# Plot hazard ratios
cph.plot()
plt.title("Cox Proportional Hazards - Hazard Ratios")
plt.show()


Feature Importance with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Prepare dataset
X = conflict_copy.select_dtypes(include=['float64', 'int64']).drop('best', axis=1, errors='ignore')
y = conflict_copy['best']  # Replace 'best' with any target column

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X.fillna(0), y.fillna(0))  # Replace missing values for simplicity

# Feature importance
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
feature_importances.sort_values().plot(kind='barh', figsize=(10, 6), title="Feature Importance")
plt.show()


#4. Predictive analysis

In [ ]:
model_scores={}

Linear regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

model_scores1 = {}  # Dictionary to store model performance

# Features and Target
features = ['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown']
target = 'best_est'

X = df1[features].fillna(0)
y = df1[target].fillna(0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

# Metrics
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
mape_lr = np.mean(np.abs((y_test - y_pred_lr) / y_test.replace(0, np.nan))) * 100  # Avoid division by zero

# Store in dictionary
model_scores['Linear Regression'] = {
    "MSE": mse_lr,
    "RMSE": rmse_lr,
    "MAE": mae_lr,
    "R2 Score": r2_lr,
    "MAPE": mape_lr
}

print("Linear Regression Metrics:")
for k, v in model_scores['Linear Regression'].items():
    print(f"{k}: {v:.4f}")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, label="Actual", color="blue", marker='o')
plt.plot(y_pred_lr, label="Predicted", color="orange", marker='x')
plt.title("Linear Regression: Actual vs Predicted")
plt.xlabel("Index")
plt.ylabel("Number of Fatalities (best_est)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

# Metrics
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
mape_rf = np.mean(np.abs((y_test - y_pred_rf) / y_test.replace(0, np.nan))) * 100  # Avoid div by zero

# Store in dictionary
model_scores['Random Forest'] = {
    "MSE": mse_rf,
    "RMSE": rmse_rf,
    "MAE": mae_rf,
    "R2 Score": r2_rf,
    "MAPE": mape_rf
}

print("Random Forest Metrics:")
for k, v in model_scores['Random Forest'].items():
    print(f"{k}: {v:.4f}")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, label="Actual", color="blue", marker='o')
plt.plot(y_pred_rf, label="Predicted", color="orange", marker='x')
plt.title("Random Forest: Actual vs Predicted")
plt.xlabel("Index")
plt.ylabel("Number of Fatalities (best_est)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_test)

# Metrics
mse_gb = mean_squared_error(y_test, y_pred_gb)
rmse_gb = np.sqrt(mse_gb)
mae_gb = mean_absolute_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)
mape_gb = np.mean(np.abs((y_test - y_pred_gb) / y_test.replace(0, np.nan))) * 100  # Avoid division by zero

# Store in dictionary
model_scores['Gradient Boosting'] = {
    "MSE": mse_gb,
    "RMSE": rmse_gb,
    "MAE": mae_gb,
    "R2 Score": r2_gb,
    "MAPE": mape_gb
}

print("Gradient Boosting Metrics:")
for k, v in model_scores['Gradient Boosting'].items():
    print(f"{k}: {v:.4f}")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, label="Actual", color="blue", marker='o')
plt.plot(y_pred_gb, label="Predicted", color="orange", marker='x')
plt.title("Gradient Boosting: Actual vs Predicted")
plt.xlabel("Index")
plt.ylabel("Number of Fatalities (best_est)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


xgboost

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

# Metrics
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
mape_xgb = np.mean(np.abs((y_test - y_pred_xgb) / y_test.replace(0, np.nan))) * 100  # Avoid div by zero

# Store in dictionary
model_scores['XGBoost'] = {
    "MSE": mse_xgb,
    "RMSE": rmse_xgb,
    "MAE": mae_xgb,
    "R2 Score": r2_xgb,
    "MAPE": mape_xgb
}

print("XGBoost Metrics:")
for k, v in model_scores['XGBoost'].items():
    print(f"{k}: {v:.4f}")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, label="Actual", color="blue", marker='o')
plt.plot(y_pred_xgb, label="Predicted", color="orange", marker='x')
plt.title("XGBoost: Actual vs Predicted")
plt.xlabel("Index")
plt.ylabel("Number of Fatalities (best_est)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


ARIMA

In [ ]:
ts_data = df1.groupby('year')['best_est'].sum()
ts_data = ts_data.sort_index()

from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Fit ARIMA (you may need to adjust the (p,d,q) order based on AIC/BIC later)
model = ARIMA(ts_data, order=(1,1,1))
arima_result = model.fit()

# Predict for the next 5 years
forecast_years = list(range(ts_data.index[-1] + 1, ts_data.index[-1] + 6))
forecast_arima = arima_result.forecast(steps=5)

# Combine with historical data
ts_forecast = pd.Series(forecast_arima.values, index=forecast_years)

# Plot actual vs forecasted
plt.figure(figsize=(10, 5))
plt.plot(ts_data.index, ts_data.values, label='Actual', color='blue', marker='o')
plt.plot(ts_forecast.index, ts_forecast.values, label='Forecast (ARIMA)', color='orange', marker='x')
plt.title('ARIMA Model - Actual vs Forecasted Fatalities')
plt.xlabel('Year')
plt.ylabel('Total Fatalities')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Align actual and predicted for evaluation (use last 5 years of actuals)
actual_arima = ts_data[-5:]
predicted_arima = arima_result.predict(start=actual_arima.index[0], end=actual_arima.index[-1])

# Metrics
mse_arima = mean_squared_error(actual_arima, predicted_arima)
rmse_arima = np.sqrt(mse_arima)
mae_arima = mean_absolute_error(actual_arima, predicted_arima)
r2_arima = r2_score(actual_arima, predicted_arima)
mape_arima = np.mean(np.abs((actual_arima - predicted_arima) / np.where(actual_arima == 0, np.nan, actual_arima))) * 100

# Store in model_scores1
model_scores["ARIMA (Yearly)"] = {
    "MSE": mse_arima,
    "RMSE": rmse_arima,
    "MAE": mae_arima,
    "R2 Score": r2_arima,
    "MAPE": mape_arima
}

# Print
print("\nARIMA (Yearly) Metrics Stored in model_scores1:")
for k, v in model_scores["ARIMA (Yearly)"].items():
    print(f"{k}: {v:.4f}")


SARIMA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Step 1️⃣: Use conflict_copy dataset
df = conflict_copy.copy()
df['conflict_occurred'] = 1  # Each row represents a conflict
conflicts_per_year = df.groupby('year')['conflict_occurred'].count()
conflicts_per_year.index = conflicts_per_year.index.astype(int)

# Step 2️⃣: Check for stationarity using ADF test
def adf_test(timeseries):
    result = adfuller(timeseries)
    print("ADF Statistic:", result[0])
    print("p-value:", result[1])
    if result[1] <= 0.05:
        print("✅ The data is stationary.")
    else:
        print("❌ The data is NOT stationary.")

adf_test(conflicts_per_year)

# Step 3️⃣: Seasonal decomposition (for insight)
decomposition = seasonal_decompose(conflicts_per_year, model="additive", period=5)
decomposition.plot()
plt.show()

# Step 4️⃣: Train-test split for time series (last 5 years as test)
train = conflicts_per_year[:-5]
test = conflicts_per_year[-5:]

# Step 5️⃣: Fit SARIMA model (parameters are manually chosen)
sarima_model = SARIMAX(train,
                       order=(1, 1, 1),
                       seasonal_order=(1, 1, 1, 5),
                       enforce_stationarity=False,
                       enforce_invertibility=False)
sarima_fit = sarima_model.fit()
print(sarima_fit.summary())

# Step 6️⃣: Forecast for test period
sarima_pred = sarima_fit.forecast(steps=len(test))

# Step 7️⃣: Evaluate SARIMA model
mse = mean_squared_error(test, sarima_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test, sarima_pred)
r2 = r2_score(test, sarima_pred)
mape = np.mean(np.abs((test - sarima_pred) / test)) * 100

# Step 8️⃣: Store and display metrics
model_scores["SARIMA_1"] = {
    "MSE": mse,
    "RMSE": rmse,
    "MAE": mae,
    "R2 Score": r2,
    "MAPE": mape
}

print("\n📊 SARIMA Evaluation Metrics:")
for metric, value in model_scores["SARIMA_1"].items():
    print(f"{metric}: {value:.4f}")

# Step 9️⃣: Plot actual vs predicted for test period
plt.figure(figsize=(10, 5))
plt.plot(conflicts_per_year, label="Actual Conflicts", marker="o")
plt.plot(test.index, sarima_pred, label="Predicted (Test)", linestyle="--", color="red", marker="x")
plt.axvline(test.index[0], color="gray", linestyle="--", label="Train/Test Split")
plt.xlabel("Year")
plt.ylabel("Number of Conflicts")
plt.title("SARIMA Model: Actual vs Predicted Conflicts")
plt.legend()
plt.tight_layout()
plt.show()

# Step 🔟 (Optional): Forecast future 5 years
future_forecast = sarima_fit.forecast(steps=5)
future_years = np.arange(conflicts_per_year.index[-1] + 1, conflicts_per_year.index[-1] + 6)

plt.figure(figsize=(10, 5))
plt.plot(conflicts_per_year, label="Historical Conflicts", marker="o")
plt.plot(future_years, future_forecast, label="Forecasted Conflicts", linestyle="--", color="green", marker="s")
plt.axvline(conflicts_per_year.index[-1], color="gray", linestyle="--", label="Forecast Start")
plt.xlabel("Year")
plt.ylabel("Number of Conflicts")
plt.title("Forecast for Next 5 Years using SARIMA")
plt.legend()
plt.tight_layout()
plt.show()


LSTM

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
df = conflict_copy.copy()

# Aggregate data by year
df['conflict_occurred'] = 1
conflicts_per_year = df.groupby('year')['conflict_occurred'].count()

# Convert index to integer years
conflicts_per_year.index = conflicts_per_year.index.astype(int)

# Normalize data (LSTMs work better with scaled data)
scaler = MinMaxScaler(feature_range=(0, 1))
conflicts_scaled = scaler.fit_transform(conflicts_per_year.values.reshape(-1, 1))

def create_sequences(data, time_steps=3):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 3  # Use last 3 years to predict the next
X, y = create_sequences(conflicts_scaled, time_steps)

# Split into train (80%) and test (20%)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape input for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(time_steps, 1)),
    LSTM(50, activation='relu'),
    Dense(1)  # Output layer
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test), verbose=1)

y_pred = model.predict(X_test)

# Inverse transform predictions to original scale
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot Actual vs. Predicted
plt.figure(figsize=(10, 5))
plt.plot(range(len(y_test_inv)), y_test_inv, label="Actual Conflicts", marker="o")
plt.plot(range(len(y_pred_inv)), y_pred_inv, label="Predicted Conflicts", marker="o", linestyle="dashed", color="red")
plt.xlabel("Time")
plt.ylabel("Number of Conflicts")
plt.title("LSTM Forecasting for Future Conflicts")
plt.legend()
plt.show()

# Get last 'time_steps' data points
last_values = conflicts_scaled[-time_steps:].reshape(1, time_steps, 1)

future_preds = []
num_years = 5  # Forecast next 5 years

for _ in range(num_years):
    next_pred = model.predict(last_values)  # Predict next value
    future_preds.append(next_pred[0, 0])  # Store prediction
    last_values = np.append(last_values[:, 1:, :], next_pred.reshape(1, 1, 1), axis=1)  # ✅ Fix dimension

# Convert predictions back to original scale
future_preds_inv = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))

# Create future years
future_years = np.arange(conflicts_per_year.index[-1] + 1, conflicts_per_year.index[-1] + 1 + num_years)

# Plot Future Predictions
plt.figure(figsize=(10, 5))
plt.plot(conflicts_per_year.index, conflicts_per_year, label="Actual Conflicts", marker="o")
plt.plot(future_years, future_preds_inv, label="Forecasted Conflicts", marker="o", linestyle="dashed", color="red")
plt.axvline(conflicts_per_year.index[-1], color="gray", linestyle="--", label="Forecast Start")
plt.xlabel("Year")
plt.ylabel("Number of Conflicts")
plt.title("LSTM Future Forecast")
plt.legend()
plt.show()

y_pred = model.predict(X_test)

# Convert back to original scale
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # Avoid division by zero

# Print Metrics
print(f"🔹 Mean Squared Error (MSE): {mse:.2f}")
print(f"🔹 Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"🔹 Mean Absolute Error (MAE): {mae:.2f}")
print(f"🔹 R² Score: {r2:.4f}")

model_scores["LSTM"] = {
    "MSE": mse,
    "RMSE": rmse,
    "MAE": mae,
    "R2 Score": r2,
    "MAPE": mape
}

In [ ]:
# Group by year and sort
ts_data = df1.groupby('year')['best_est'].sum()
ts_data = ts_data.sort_index()
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Scale data (reshape to 2D first)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(ts_data.values.reshape(-1, 1))

# Prepare sequences (e.g., 5 years lookback)
X, y = [], []
for i in range(5, len(scaled_data)):
    X.append(scaled_data[i-5:i])
    y.append(scaled_data[i])

X, y = np.array(X), np.array(y)

# Reshape X for LSTM: (samples, time_steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(64, activation='relu', input_shape=(X.shape[1], 1)))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mse')

# Train
model_lstm.fit(X, y, epochs=100, verbose=0)

# Predict
y_pred_lstm = model_lstm.predict(X)
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)
actual_lstm = scaler.inverse_transform(y)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(actual_lstm, label='Actual', color='blue')
plt.plot(y_pred_lstm, label='LSTM Predicted', color='orange', linestyle='--')
plt.title("LSTM: Actual vs Predicted Fatalities")
plt.xlabel("Index (Time Steps)")
plt.ylabel("Fatalities")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Evaluation metrics
mse_lstm = mean_squared_error(actual_lstm, y_pred_lstm)
rmse_lstm = np.sqrt(mse_lstm)
mae_lstm = mean_absolute_error(actual_lstm, y_pred_lstm)
r2_lstm = r2_score(actual_lstm, y_pred_lstm)
mape_lstm = np.mean(np.abs((actual_lstm - y_pred_lstm) / np.where(actual_lstm == 0, np.nan, actual_lstm))) * 100

# Store in model_scores1
model_scores["LSTM (Yearly)"] = {
    "MSE": mse_lstm,
    "RMSE": rmse_lstm,
    "MAE": mae_lstm,
    "R2 Score": r2_lstm,
    "MAPE": mape_lstm
}

# Print
print("\nLSTM (Yearly) Metrics Stored in model_scores1:")
for k, v in model_scores["LSTM (Yearly)"].items():
    print(f"{k}: {v:.4f}")


Prophet

In [ ]:
# Prepare monthly data
df_time = df_clean[["date_start", "best"]].copy()
df_time = df_time.dropna()
df_time["date_start"] = pd.to_datetime(df_time["date_start"])
df_time = df_time.groupby(pd.Grouper(key="date_start", freq="M")).sum().reset_index()
df_time.columns = ["ds", "y"]  # Prophet requires these column names

# Check for missing time points
df_time = df_time.set_index("ds").asfreq("M").reset_index()
missing_months = df_time["y"].isna().sum()
print(f"Missing months in time series: {missing_months}")

# Fill missing months with 0 if any
df_time["y"] = df_time["y"].fillna(0)

# Check for spikes (print summary)
print(df_time["y"].describe())

# Cap outliers to the 99th percentile
q99 = df_time["y"].quantile(0.99)
df_time["y"] = np.where(df_time["y"] > q99, q99, df_time["y"])

# Train/test split again after fixing
train_size = int(len(df_time) * 0.8)
train = df_time.iloc[:train_size]
test = df_time.iloc[train_size:]

# Re-train Prophet
from prophet import Prophet
model = Prophet()
model.fit(train)

# Predict
future = model.make_future_dataframe(periods=len(test), freq='M')
forecast = model.predict(future)

# Evaluate
predicted = forecast[['ds', 'yhat']].iloc[-len(test):].reset_index(drop=True)
actual = test.reset_index(drop=True)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Evaluate
mae = mean_absolute_error(actual["y"], predicted["yhat"])
mse = mean_squared_error(actual["y"], predicted["yhat"])
rmse = np.sqrt(mse)
r2 = r2_score(actual["y"], predicted["yhat"])
mape = np.mean(np.abs((actual["y"] - predicted["yhat"]) / actual["y"].replace(0, np.nan))) * 100  # Safe division

# Store in dictionary
model_scores["Prophet (Monthly)"] = {
    "MSE": mse,
    "RMSE": rmse,
    "MAE": mae,
    "R2 Score": r2,
    "MAPE": mape
}

print("\nProphet (Monthly) Metrics Stored in model_scores1:")
for k, v in model_scores["Prophet (Monthly)"].items():
    print(f"{k}: {v:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(actual["ds"], actual["y"], label="Actual", marker='o')
plt.plot(predicted["ds"], predicted["yhat"], label="Predicted", marker='x')
plt.title("Prophet Forecast vs Actual Fatalities (After Cleaning)")
plt.xlabel("Date")
plt.ylabel("Fatalities")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Storing model performance

In [ ]:
model_scores

In [ ]:
df_scores = pd.DataFrame(model_scores).T
print("\nModel Performance Comparison:\n")
df_scores

#N-sampling method to compare the predictive analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import pi

# Your data in dictionary format
model_results = model_scores

# Convert to DataFrame
df = pd.DataFrame.from_dict(model_results, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Model'}, inplace=True)

# Display the data
print("Model Performance Metrics:")
print(df.to_string(index=False))

# Visualization 1: Bar plot for each metric
metrics = ['MSE', 'RMSE', 'MAE', 'R2 Score', 'MAPE']
plt.figure(figsize=(15, 20))

for i, metric in enumerate(metrics, 1):
    plt.subplot(3, 2, i)
    if metric == 'R2 Score':
        # For R2, higher is better
        sorted_df = df.sort_values(metric, ascending=False)
        bars = plt.barh(sorted_df['Model'], sorted_df[metric], color='skyblue')
        plt.title(f'{metric} (Higher is better)')
        # Add value labels
        for bar in bars:
            width = bar.get_width()
            plt.text(width, bar.get_y() + bar.get_height()/2, f'{width:.3f}',
                    ha='left', va='center')
    else:
        # For error metrics, lower is better
        sorted_df = df[df[metric].notna()].sort_values(metric)
        bars = plt.barh(sorted_df['Model'], sorted_df[metric], color='lightcoral')
        plt.title(f'{metric} (Lower is better)')
        # Add value labels
        for bar in bars:
            width = bar.get_width()
            plt.text(width, bar.get_y() + bar.get_height()/2, f'{width:.3f}',
                    ha='left', va='center')

    plt.xlabel(metric)
    plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.suptitle('Model Performance Comparison', y=1.02, fontsize=16)
plt.show()

# Visualization 2: Radar chart for top models
# Select top models based on R2 Score (excluding negative values)
top_models = df[df['R2 Score'] > 0].sort_values('R2 Score', ascending=False).head(4)['Model'].values

# Normalize metrics for radar chart (except R2 which is already normalized)
df_normalized = df.copy()
for metric in ['MSE', 'RMSE', 'MAE', 'MAPE']:
    min_val = df[metric].min()
    df_normalized[metric] = df[metric] / min_val

df_radar = df_normalized[df_normalized['Model'].isin(top_models)]

# Number of variables
categories = ['MSE', 'RMSE', 'MAE', 'R2 Score', 'MAPE']
N = len(categories)

# Create radar chart
plt.figure(figsize=(8, 8))
ax = plt.subplot(111, polar=True)

# Draw one axe per variable + add labels
plt.xticks(np.linspace(0, 2*pi, N, endpoint=False), categories)
ax.set_rlabel_position(0)
plt.yticks([1, 2, 3, 4, 5], ["1x", "2x", "3x", "4x", "5x"], color="grey", size=7)
plt.ylim(0, 5)

# Plot each model
for idx, row in df_radar.iterrows():
    values = row[categories].values.flatten().tolist()
    values += values[:1]  # Complete the loop
    angles = np.linspace(0, 2*pi, N, endpoint=False).tolist()
    angles += angles[:1]

    ax.plot(angles, values, linewidth=2, linestyle='solid', label=row['Model'])
    ax.fill(angles, values, alpha=0.1)

plt.title('Top Models Comparison (Normalized Metrics)', size=16, y=1.1)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.show()

# Determine the best model based on multiple criteria
# Rank each model for each metric and sum the ranks
# For error metrics (MSE, RMSE, MAE, MAPE), lower is better
# For R2, higher is better

df_rank = df.copy()

# Rank models for each metric
for metric in ['MSE', 'RMSE', 'MAE', 'MAPE']:
    df_rank[metric+'_rank'] = df_rank[metric].rank(ascending=True)

df_rank['R2_rank'] = df_rank['R2 Score'].rank(ascending=False)

# Calculate total rank (sum of individual ranks)
df_rank['Total_rank'] = (df_rank['MSE_rank'] + df_rank['RMSE_rank'] +
                         df_rank['MAE_rank'] + df_rank['R2_rank'] +
                         df_rank['MAPE_rank'].fillna(len(df_rank)))  # Fill NA with worst rank

# Sort by total rank
df_rank = df_rank.sort_values('Total_rank')

print("\nModel Rankings (Lower total rank is better):")
print(df_rank[['Model', 'Total_rank']].to_string(index=False))

# Best model
best_model = df_rank.iloc[0]['Model']
print(f"\nBest Model Based on Combined Metrics: {best_model}")

# Show top 3 models
print("\nTop 3 Models:")
print(df_rank[['Model', 'Total_rank']].head(3).to_string(index=False))

# Additional insights
print("\nAdditional Insights:")
print("- XGBoost and Gradient Boosting show excellent performance with R2 > 0.99")
print("- Tree-based models (Random Forest, XGBoost, Gradient Boosting) dominate the top positions")
print("- Time series models (ARIMA, SARIMA, Prophet) perform poorly on this dataset")
print("- LSTM has low error metrics but negative R2, suggesting potential issues with the model")

In [ ]:
print("\nModel Rankings (Lower total rank is better):")
styled_df = df_rank[['Model', 'Total_rank']].style.hide()
styled_df

In [ ]:
print("\nTop 3 Models:")
styled_df = df_rank[['Model', 'Total_rank']].head(3).style.hide()
styled_df

#5. Prescriptive Analysis

Adding Socio_economic factors

In [ ]:
socio_df = pd.read_csv("https://raw.githubusercontent.com/bhadri-Raj-T/eda_project/refs/heads/main/socio_economic_factors.csv")

In [ ]:
# Keep only the indicator name and year columns
years = [col for col in socio_df.columns if col.startswith("19") or col.startswith("20")]
socio_df_clean = socio_df[['Series Name'] + years].copy()

# Melt the dataset to long format: one row per indicator per year
socio_long = socio_df_clean.melt(id_vars='Series Name', var_name='year', value_name='value')

# Extract just the 4-digit year from strings like "1990 [YR1990]"
socio_long['year'] = socio_long['year'].str.extract(r'(\d{4})').astype(int)

# Remove '..' placeholders and convert to numeric
socio_long = socio_long[~socio_long['value'].isin(['..'])]
socio_long['value'] = pd.to_numeric(socio_long['value'], errors='coerce')

# Pivot the table to get one row per year, with each indicator as a column
socio_wide = socio_long.pivot(index='year', columns='Series Name', values='value').reset_index()

# Preview the reshaped data
socio_wide.head()


In [ ]:
socio_wide.info()

In [ ]:
# Copy the original to preserve it
socio_cleaned = socio_wide.copy()

# Drop columns with more than 50% missing values
threshold = 0.5  # 50%
min_count = int(threshold * len(socio_cleaned))
socio_cleaned = socio_cleaned.dropna(axis=1, thresh=min_count)

# For remaining columns, apply interpolation or fill with mean
for col in socio_cleaned.columns:
    if socio_cleaned[col].isnull().sum() > 0:
        if socio_cleaned[col].dtype in ['float64', 'int64']:
            # Use linear interpolation for time series
            socio_cleaned[col] = socio_cleaned[col].interpolate(method='linear', limit_direction='both')
            # Fill remaining NaNs (if any) with column mean
            socio_cleaned[col] = socio_cleaned[col].fillna(socio_cleaned[col].mean())

# Confirm missing values are handled
print(socio_cleaned.isnull().sum())
socio_cleaned.head()

In [ ]:
socio_cleaned.columns

In [ ]:
df1.columns

In [ ]:
# Make sure 'df1' (your conflict dataset) is ready
# Group by year to get total fatalities
fatalities_per_year = df1.groupby('year')['best_est'].sum().reset_index()

# Merge with socio-economic data on 'year'
combined_df1 = pd.merge(socio_cleaned, fatalities_per_year, on='year', how='inner')

# Rename fatalities column for clarity
combined_df1.rename(columns={'best_est': 'fatalities'}, inplace=True)

# Preview merged dataset
combined_df1.head()


In [ ]:
# Compute correlation matrix
correlation_matrix = combined_df1.corr(numeric_only=True)

# Show top correlations with fatalities
correlation_with_fatalities = correlation_matrix['fatalities'].sort_values(ascending=False)
print("Correlations with Conflict Fatalities:\n")
print(correlation_with_fatalities)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set plot size
plt.figure(figsize=(12, 8))

# Create the heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True)
plt.title("Correlation Between Socio-Economic Indicators and Conflict Fatalities", fontsize=10)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
# Example: GDP growth vs Fatalities
plt.figure(figsize=(8, 5))
sns.scatterplot(data=combined_df1, x='GDP growth (annual %)', y='fatalities')
plt.title("GDP Growth vs Conflict Fatalities")
plt.xlabel("GDP Growth (%)")
plt.ylabel("Total Fatalities")
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
sns.scatterplot(data=combined_df1, x='Vulnerable employment, total (% of total employment) (modeled ILO estimate)', y='fatalities')
plt.title("Vulnerable employment vs Conflict Fatalities")
plt.xlabel("Vulnerable employment, total (% of total employment)")
plt.ylabel("Total Fatalities")
plt.grid(True)
plt.show()


plt.figure(figsize=(8, 5))
sns.scatterplot(data=combined_df1, x='School enrollment, secondary (% gross)', y='fatalities')
plt.title("School enrollment vs Conflict Fatalities")
plt.xlabel("School enrollment, secondary (% gross)")
plt.ylabel("Total Fatalities")
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
sns.scatterplot(data=combined_df1, x='Children out of school (% of primary school age)', y='fatalities')
plt.title("Children out of school  vs Conflict Fatalities")
plt.xlabel("Children out of school (% of primary school age)")
plt.ylabel("Total Fatalities")
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
sns.scatterplot(data=combined_df1, x='Primary education, pupils', y='fatalities')
plt.title("Primary education, pupils  vs Conflict Fatalities")
plt.xlabel("Primary education, pupils")
plt.ylabel("Total Fatalities")
plt.grid(True)
plt.show()

Socio_Economin_Factors

In [ ]:
# Load the dataset
sef2_df1 = pd.read_csv("https://raw.githubusercontent.com/bhadri-Raj-T/eda_project/refs/heads/main/sef2.csv")

# Step 1: Keep only 'Series Name' and year columns
year_cols = [col for col in sef2_df1.columns if col.startswith("19") or col.startswith("20")]
sef2_clean = sef2_df1[['Series Name'] + year_cols].copy()

# Step 2: Melt the dataset to long format
sef2_long = sef2_clean.melt(id_vars='Series Name', var_name='year', value_name='value')

# Step 3: Clean year column and convert to numeric
sef2_long['year'] = sef2_long['year'].str.extract(r'(\d{4})').astype(int)

# Step 4: Remove placeholders like '..' and convert to float
sef2_long = sef2_long[~sef2_long['value'].isin(['..'])]
sef2_long['value'] = pd.to_numeric(sef2_long['value'], errors='coerce')

# Step 5: Pivot to wide format (one row per year)
sef2_wide = sef2_long.pivot(index='year', columns='Series Name', values='value').reset_index()

# Show initial structure
print(sef2_wide.info())

# Step 6: Drop columns with more than 50% missing data
threshold = 0.5  # 50% threshold
min_non_na = int(threshold * len(sef2_wide))
sef2_wide = sef2_wide.dropna(thresh=min_non_na, axis=1)

# Step 7: Interpolate remaining missing values, fallback to mean
for col in sef2_wide.columns:
    if col != 'year' and sef2_wide[col].isnull().sum() > 0:
        # Interpolate over years
        sef2_wide[col] = sef2_wide[col].interpolate(method='linear', limit_direction='both')
        # Fill any remaining NaNs with column mean
        sef2_wide[col] = sef2_wide[col].fillna(sef2_wide[col].mean())

# Step 8: Confirm cleanup
print("Missing values after cleaning:")
print(sef2_wide.isnull().sum())
sef2_wide.head()


fatalities

In [ ]:
# Step 1: Aggregate fatalities per year from conflict dataset
fatalities_by_year = df1.groupby('year')['best_est'].sum().reset_index()
fatalities_by_year.rename(columns={'best_est': 'fatalities'}, inplace=True)

# Step 2: Merge with socio-economic data on 'year'
merged_sef2 = pd.merge(sef2_wide, fatalities_by_year, on='year', how='inner')

# Preview merged dataset
merged_sef2.head()

In [ ]:
# Compute Spearman correlation matrix
spearman_corr = merged_sef2.corr(method='spearman')

# Show top correlations with fatalities
print(spearman_corr['fatalities'].sort_values(ascending=False))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 10))
sns.heatmap(spearman_corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Spearman Correlation Heatmap: Socio-Economic Indicators vs Fatalities")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


Linear programming

In [ ]:
# Check for missing or zero values
print(conflict_copy[['deaths_a', 'deaths_b', 'event_clarity', 'number_of_sources']].describe())

# Drop rows with missing or zero values
conflict_copy = conflict_copy.dropna(subset=['deaths_a', 'deaths_b', 'event_clarity', 'number_of_sources'])
conflict_copy = conflict_copy[(conflict_copy['deaths_a'] > 0) & (conflict_copy['deaths_b'] > 0)]
conflict_copy['event_clarity']=conflict_copy['event_clarity'].astype(int)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Example: Ensure your DataFrame 'conflict_copy' is loaded beforehand
# For example: conflict_copy = pd.read_csv("conflict_data.csv")

# Clean the data: Drop rows with NaN in required columns
conflict_copy = conflict_copy.dropna(subset=['deaths_a', 'deaths_b', 'event_clarity', 'number_of_sources'])

# Convert necessary columns to numeric (handle categorical issues)
conflict_copy["deaths_a"] = pd.to_numeric(conflict_copy["deaths_a"], errors='coerce')
conflict_copy["deaths_b"] = pd.to_numeric(conflict_copy["deaths_b"], errors='coerce')
conflict_copy["event_clarity"] = pd.to_numeric(conflict_copy["event_clarity"], errors='coerce')
conflict_copy["number_of_sources"] = pd.to_numeric(conflict_copy["number_of_sources"], errors='coerce')

# Filter only positive death counts
conflict_copy = conflict_copy[(conflict_copy['deaths_a'] > 0) & (conflict_copy['deaths_b'] > 0)]

# Define the LP problem
model = LpProblem(name="conflict_resource_optimization", sense=LpMinimize)

# Decision variables: Allocation of resources to each conflict zone
zones = conflict_copy.index
allocation = {z: LpVariable(name=f"alloc_{z}", lowBound=0) for z in zones}

# Objective function: Minimize total deaths
model += lpSum((conflict_copy.loc[z, "deaths_a"] + conflict_copy.loc[z, "deaths_b"]) * allocation[z] for z in zones), "Minimize_Deaths"

# Constraint: Total resources cannot exceed 2x the sum of clarity and sources
total_resources = 2 * (conflict_copy["event_clarity"].sum() + conflict_copy["number_of_sources"].sum())
model += lpSum(allocation[z] for z in zones) <= total_resources, "Total_Resource_Limit"

# Constraint: Minimum allocation for each zone
for z in zones:
    min_alloc = 0.1 * (conflict_copy.loc[z, "deaths_a"] + conflict_copy.loc[z, "deaths_b"])
    model += allocation[z] >= min_alloc, f"Min_Allocation_{z}"

# Solve the model
model.solve()

# Print results
print("Optimal Resource Allocation:")
for z in zones:
    print(f"Conflict Zone {z}: {allocation[z].varValue:.2f}")

print(f"\nTotal Minimized Deaths: {model.objective.value():.2f}")

# Extract and plot optimal allocations
optimal_allocations = [allocation[z].varValue for z in zones]

plt.figure(figsize=(12, 6))
plt.bar(zones.astype(str), optimal_allocations, color='green')
plt.xlabel("Conflict Zone")
plt.ylabel("Optimal Resource Allocation")
plt.title("Optimal Resource Allocation Across Conflict Zones")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


Optimization of resourses

In [ ]:
from scipy.optimize import minimize

# Ensure data is clean
conflict_copy = conflict_copy.dropna(subset=['deaths_a', 'deaths_b', 'event_clarity', 'number_of_sources'])
conflict_copy = conflict_copy[(conflict_copy['deaths_a'] > 0) & (conflict_copy['deaths_b'] > 0)]

# Define the objective function
def objective(x):
    deaths_a = conflict_copy['deaths_a'].values
    deaths_b = conflict_copy['deaths_b'].values
    # Maximize reduction in deaths (negative sign for maximization)
    return -np.sum((deaths_a + deaths_b) * x)

# Define constraints
def resource_constraint(x):
    total_resources = 2 * (conflict_copy["event_clarity"].sum() + conflict_copy["number_of_sources"].sum())  # Increased resources
    return total_resources - np.sum(x)

def min_allocation_constraint(x):
    # Ensure minimum allocation of 0.1 * (deaths_a + deaths_b) for each zone
    min_allocation = 0.1 * (conflict_copy['deaths_a'].values + conflict_copy['deaths_b'].values)
    return x - min_allocation

# Initial guess (start with small allocations)
x0 = np.ones(len(conflict_copy)) * 0.1

# Bounds (allocation cannot be negative)
bounds = [(0, None) for _ in range(len(conflict_copy))]

# Constraints
constraints = [
    {'type': 'ineq', 'fun': resource_constraint},  # Total resources constraint
    {'type': 'ineq', 'fun': min_allocation_constraint}  # Minimum allocation constraint
]

# Solve the optimization problem
result = minimize(objective, x0, bounds=bounds, constraints=constraints)

# Print results
print("Optimal Resource Allocation:", result.x)
print("Total Reduction in Deaths:", -result.fun)  # Negative sign to convert back to positive

DOwhy anaylsis
Identify the causal impact of different factors on conflict intensity (measured via deaths).
Recommend prescriptive actions to reduce conflict intensity.


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from dowhy import CausalModel

# Load dataset (assuming it's already loaded as df)
# df = pd.read_csv("conflict_data.csv") # Uncomment if loading from file

# Define treatment, outcome, and confounders
treatment = "type_of_violence"
outcome = "best"
confounders = ["region", "country", "side_a", "side_b", "latitude", "longitude"]

# Build the Causal Model
model = CausalModel(
    data=conflict_copy,
    treatment=treatment,
    outcome=outcome,
    common_causes=confounders
)

# Visualize the Causal Graph (DAG)
plt.figure(figsize=(10,6))
model.view_model(layout="dot")
plt.show()


Deep Q-Learning (DQN) → Stable-Baselines3, TensorFlow

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

# Create a vectorized environment (Colab-friendly)
env = make_vec_env("CartPole-v1", n_envs=1)

# Train DQN Model
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Save & Load the Model
model.save("dqn_cartpole")
del model  # Remove from memory
model = DQN.load("dqn_cartpole")

# Test the trained model
obs = env.reset()
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()


In [ ]:
import gym
import numpy as np
from gym import spaces
from stable_baselines3 import DQN
import matplotlib.pyplot as plt

# Custom Conflict Management Environment
class ConflictEnv(gym.Env):
    def __init__(self):
        super(ConflictEnv, self).__init__()

        # Action Space: 0 = No Action, 1 = Deploy Resources, 2 = Increase Surveillance
        self.action_space = spaces.Discrete(3)

        # Observation Space: Conflicts, Deaths, Event Clarity, Sources
        self.observation_space = spaces.Box(low=0, high=100, shape=(4,), dtype=np.float32)

        self.state = None

    def reset(self):
        self.state = np.array([50, 10, 5, 3], dtype=np.float32)  # Starting state
        return self.state

    def step(self, action):
        conflicts, deaths, event_clarity, sources = self.state

        # Apply Actions
        if action == 1:  # Deploy Resources
            conflicts = max(0, conflicts - 5)
            deaths = max(0, deaths - 2)
        elif action == 2:  # Increase Surveillance
            event_clarity = min(100, event_clarity + 2)
            sources = min(100, sources + 1)

        # Reward: lower conflicts & deaths + better clarity/sources
        reward = (100 - conflicts) + (50 - deaths) + (event_clarity * 2) + (sources * 3)

        self.state = np.array([conflicts, deaths, event_clarity, sources], dtype=np.float32)
        done = conflicts <= 0

        return self.state, reward, done, {}

    def render(self, mode='human'):
        print(f"State: Conflicts={self.state[0]}, Deaths={self.state[1]}, Clarity={self.state[2]}, Sources={self.state[3]}")

# Create Environment
env = ConflictEnv()

# Train DQN Model
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Test the Trained Model
obs = env.reset()
states, rewards_log = [], []

for step in range(20):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    env.render()
    states.append(obs.copy())
    rewards_log.append(reward)
    if done:
        break

# Visualization: Rewards over Steps
plt.figure(figsize=(10, 4))
plt.plot(rewards_log, marker='o', color='darkgreen')
plt.title("Reward Over Time Steps")
plt.xlabel("Step")
plt.ylabel("Reward")
plt.grid(True)
plt.tight_layout()
plt.show()

Absolutely — let’s break down your results and extract **data-driven insights** along with **actionable policy recommendations**.

---

# **Insights from Spearman and Pearson Correlations**

## 1. **Strong Positive Correlation with Fatalities**
###  *Battle-related deaths (r = 0.63, Spearman)*  
This is expected — UCDP's `best_est` and battle-related deaths should overlap. It confirms your target variable is consistent with battle metrics.

**Use this as a validation point** in your methodology:  
"The strong correlation between `best_est` and battle-related deaths validates that our dataset reflects actual conflict intensity."

---

## 2. **Military & Defense-Related Indicators**
###  *Arms Exports (r = 0.30, Spearman)*  
###  *Military Expenditure (% of GDP) (r = 0.29, Spearman)*  
 Countries with higher military activity may face higher conflict fatalities, suggesting increased militarization could fuel conflict escalation — especially if not paired with peacebuilding efforts.

**Insight**:  
*"A rise in military spending and arms exports appears moderately linked with increased conflict fatalities. While defense may be essential, unchecked militarization could worsen internal tensions."*

**Policy Recommendations**:
- Implement transparency in defense budgeting.
- Balance military expenditure with peacebuilding investments (e.g., conflict resolution education, local peace councils).
- Promote arms trade regulation with conflict-sensitive policies.

---

## 3. **Demographic Indicators**
###  *Population Ages 0–14, Male (r = 0.17)*  
 A younger, male-heavy population is sometimes correlated with unrest if youth are unemployed or disenfranchised — often referred to as the **"youth bulge" hypothesis**.

 **Insight**:  
"Regions with large young male populations may require targeted interventions, as youth marginalization can be a risk factor for conflict."

**Policy Recommendations**:
- Invest in youth education and vocational programs.
- Promote inclusive youth political participation and representation.
- Launch youth employment schemes in high-risk regions.

---

## 4. **Negative Correlation with Fatalities (Risk Reducing Factors)**

###  *Refugee Population by Country (-0.43, Spearman)*  
India’s role as a host country might align with regional humanitarian stability or international cooperation. Lower fatalities where refugee policies are in place might be a signal of **international engagement**.

**Insight**:  
*"Hosting refugees and participating in international humanitarian efforts may align with reduced conflict fatalities."*

**Policy Recommendations**:
- Strengthen refugee protection policies and align with UNHCR best practices.
- Leverage international peacebuilding aid tied to refugee support programs.

---

## 5. **Education & Employment (from Pearson correlations)**

###  *Vulnerable Employment (r = 0.34)*  
###  *Low School Enrollment (r = -0.12 to -0.18)*  
High vulnerable employment (e.g., informal jobs with no security) and low school enrollment are lightly correlated with fatalities.

 **Insight**:  
*"Socio-economic instability, particularly lack of secure employment and low education access, correlates with greater conflict fatality rates."*

 **Policy Recommendations**:
- Expand formal job opportunities in rural/conflict-prone areas.
- Promote universal basic education and reduce drop-out rates.
- Support vocational training and skill development in underserved districts.

---

# Summary: Data-Driven Conflict Reduction Policy Strategy

| Focus Area         | Risk Factor                | Recommendation                                                                 |
|--------------------|----------------------------|--------------------------------------------------------------------------------|
| Military Spending  | High arms exports/expenditure | Increase transparency & balance with peace initiatives                          |
| Youth Demographics | Large male youth population | Target youth for jobs, education, and political inclusion                       |
| Education Access   | Low enrollment rates        | Universalize primary and secondary education                                   |
| Job Security       | Vulnerable employment       | Create formal job schemes in rural/conflict zones                              |
| International Role | Refugee population          | Strengthen refugee protection and use it as leverage for peacebuilding funding |

---


#Pre-profiling

In [ ]:
import ydata_profiling as prf
# Generate report
profile =prf.ProfileReport(conflict_copy, title="Conflict Data Pre-Profiling Report", explorative=True)
# Save the report as HTML
profile.to_file("conflict_data_profiling.html")


In [ ]:
profile